In [ ]:
# default_exp data

# data

> Read data from netdata rest api into a pandas dataframe.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
#export
import asks
import trio
import pandas as pd
import requests

In [ ]:
#export


def get_chart_list(host: str = '127.0.0.1:19999', starts_with: str = None) -> list:
    """Get list of all available charts on a `host`.  
    
    ##### Parameters:  
    - **host** `str` The host we want to get a list of available charts from.
    - **starts_with** `str` A string to filter the list of charts returns to just those that start with `starts_with`.
    
    ##### Returns:  
    - **chart_list** `list` A list of availalbe charts.
    """
    
    url = f"http://{host}/api/v1/charts"
    r = requests.get(url)
    charts = r.json().get('charts')
    chart_list = [chart for chart in charts]
    if starts_with:
        chart_list = [chart for chart in chart_list if chart.startswith(starts_with)]
    return chart_list

In [ ]:
# tests

# get some charts from london demo site
charts = get_chart_list('london.my-netdata.io', starts_with='system.')

# check just system. charts returned
assert set([chart.split('.')[0] for chart in charts]) == set(['system'])


In [ ]:
#export


async def get_chart(api_call: str, data: list, col_sep: str ='|'):
    """Get data for an individual chart.
    
    ##### Parameters:  
    - **api_call** `tuple` A tuple of (`url`,`chart`) for the url to pull data from and chart it represents.
    - **data** `list` A list for dataframes for each chart to be appended to.
    - **col_sep** `str` A character for separating chart and dimension in column names of dataframe.
    
    """
    
    url, chart, host = api_call
    r = await asks.get(url)
    r_json = r.json()
    df = pd.DataFrame(r_json['data'], columns=['time_idx'] + r_json['labels'][1:])
    df['host'] = host
    df = df.set_index(['host','time_idx']).add_prefix(f'{chart}{col_sep}')
    data.append(df)

In [ ]:
#export


async def get_charts(api_calls, col_sep='|', timeout: int = 60):
    """Create a nursey to make seperate async calls to get each chart.
    
    ##### Parameters:  
    - **api_calls** `list` A list of tuple's of [(`url`,`chart`),...] of api calls that need to be made.
    - **col_sep** `str` A character for separating chart and dimension in column names of dataframe.
    - **timeout** `int` The number of seconds for trio to [move_on_after](https://trio.readthedocs.io/en/stable/reference-core.html#trio.move_on_after).
    
    ##### Returns:  
    - **df** `pd.DataFrame` A pandas dataframe with all chart data outer joined based on time index.
    """
    
    data = []
    with trio.move_on_after(timeout):
        async with trio.open_nursery() as nursery:
            for api_call in api_calls:
                nursery.start_soon(get_chart, api_call, data, col_sep)
    df = pd.concat(data, join='outer', axis=1, sort=True)
    return df

In [ ]:
#export


def get_data(hosts: list = ['london.my-netdata.io'], charts: list = ['system.cpu'], after: int = -60, 
             before: int = 0, points: int = 0, col_sep: str = '|', numeric_only: bool = False,
             ffill: bool = True, diff: bool = False, timeout: int = 60) -> pd.DataFrame:
    """Define api calls to make and any post processing to be done.
    
    ##### Parameters:  
    - **hosts** `list` A list of hosts to pull data from.
    - **charts** `list` A list of charts to pull data for.
    - **after** `int` The timestamp or relative integer from which to pull data after.
    - **before** `int` The timestamp or relative integer from which to pull data before.
    - **points** `int` The `points` parameter to pass to the api call if need to aggregate data in some way.
    - **col_sep** `str` A character for separating chart and dimension in column names of dataframe.
    - **numeric_only** `bool` Set to true if you want to filter out any non numeric data.
    - **ffill** `bool` Set to true if you want to forward fill any null or missing values.
    - **diff** `bool` Set to true if you want to get the difference of metrics as opposed to their raw value.
    - **timeout** `int` The number of seconds for trio to [move_on_after](https://trio.readthedocs.io/en/stable/reference-core.html#trio.move_on_after).
        
    ##### Returns:  
    - **df** `pd.DataFrame` A pandas dataframe with all chart data outer joined based on time index and any post processing done.
    """
    
    # if hosts is a string make it a list of one
    if isinstance(hosts, str):
        hosts = [hosts]
        
    # if specified get all charts
    if charts == 'all':
        charts = get_chart_list(hosts[0])
    
    # define list of all api calls to be made
    api_calls = [
        (f'http://{host}/api/v1/data?chart={chart}&after={after}&before={before}&points={points}&format=json', chart, host)
        for host in hosts for chart in charts
    ] 
    # get the data
    df = trio.run(get_charts, api_calls, col_sep, timeout)
    # post process the data
    if len(hosts) == 1:
        df = df.reset_index(level=0, drop=True)
    if numeric_only:
        df = df._get_numeric_data()
    if ffill:
        df = df.ffill()
    if diff:
        df = df.diff().dropna(how='all')
    return df

In [ ]:
# examples

df = get_data('london.my-netdata.io', 'all', after=-60, before=0)
print(df.shape)
df.head()

(67, 2091)


,groups.lwrites|ulog,groups.lwrites|messagebus,groups.lwrites|unscd,groups.lwrites|bind,groups.lwrites|nogroup,groups.lwrites|ntp,groups.lwrites|postfix,groups.lwrites|mysql,groups.lwrites|www-data,groups.lwrites|netdata,...,cpu.cpu0_softnet_stat|received_rps,cpu.cpu0_softnet_stat|flow_limit_count,ipv6.icmpneighbor|InSolicits,ipv6.icmpneighbor|OutSolicits,ipv6.icmpneighbor|InAdvertisements,ipv6.icmpneighbor|OutAdvertisements,netfilter.synproxy_entries|entries,mysql_local.sort_issues|merge_passes,mysql_local.sort_issues|range,mysql_local.sort_issues|scan
time_idx,,,,,,,,,,,,,,,,,,,,,
1592302970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1592302973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1592302974,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.24441,34.15204,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1592302975,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42.27810,30.32549,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1592302976,0.253904,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.75438,35.44889,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# tests

# get some test data
test_host = 'london.my-netdata.io'
test_charts = ['system.cpu', 'system.load']
df = get_data(test_host, test_charts, after=-60, before=0, col_sep='|')

# look for some expected columns
assert 'system.load|load1' in df.columns
assert 'system.cpu|user' in df.columns
# check expected shape of data
assert str(df.shape) == '(60, 12)' or '(61, 12)'
# check that all types are float64 or int64
assert len(df.dtypes[df.dtypes != 'int64'][df.dtypes != 'float64']) == 0